## 1. Démarrage de la session Spark

In [1]:
# L'exécution de cette cellule démarre l'application Spark

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1,application_1726592999992_0002,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
%%info

ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1,application_1726592999992_0002,pyspark,idle,Link,Link,None,✔


In [3]:
import pandas as pd
import numpy as np
from PIL import Image
import numpy as np
import io
import os

from typing import Iterator
import tensorflow as tf
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras import Model
from pyspark.sql.functions import col, pandas_udf, PandasUDFType, element_at, split, udf
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.sql.types import ArrayType, FloatType
from pyspark.sql import SparkSession
from pyspark.ml.feature import PCA
import pyarrow.parquet as pq
import s3fs
import boto3
import pyarrow

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## 3.5 Définition des PATH pour charger les images <br /> et enregistrer les résultats

Dans cette version nous partons du principe que les données <br />
sont stockées dans le même répertoire que le notebook.<br />
Les images à charger sont stockée dans le dossier **Test**.<br />
Nous enregistrerons le résultat de notre traitement <br />
dans le dossier "**Results_Local**"

In [4]:
PATH = 's3://p8-data-soraya'
PATH_Data = PATH+'/jupyter/jovyan/data/Test'
PATH_Result = PATH+'/jupyter/jovyan/data/Results'
print('PATH:        '+\
      PATH+'\nPATH_Data:   '+\
      PATH_Data+'\nPATH_Result: '+PATH_Result)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

PATH:        s3://p8-data-soraya
PATH_Data:   s3://p8-data-soraya/jupyter/jovyan/data/Test
PATH_Result: s3://p8-data-soraya/jupyter/jovyan/data/Results

### 3.7.1 Chargement des données

Les images sont chargées au format binaire, ce qui offre, <br />
plus de souplesse dans la façon de prétraiter les images.

Avant de charger les images, nous spécifions que nous voulons charger <br />
uniquement les fichiers dont l'extension est **jpg**.

Nous indiquons également de charger tous les objets possibles contenus <br />
dans les sous-dossiers du dossier communiqué.

In [5]:
images = spark.read.format("binaryFile") \
  .option("pathGlobFilter", "*.jpg") \
  .option("recursiveFileLookup", "true") \
  .load(PATH_Data)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<u>Affichage des 5 premières images contenant</u> :
 - le path de l'image
 - la date et heure de sa dernière modification
 - sa longueur
 - son contenu encodé en valeur hexadécimal

In [6]:
images.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-------------------+------+--------------------+
|                path|   modificationTime|length|             content|
+--------------------+-------------------+------+--------------------+
|s3://p8-data-sora...|2024-09-11 21:10:50|  7353|[FF D8 FF E0 00 1...|
|s3://p8-data-sora...|2024-09-11 21:10:50|  7350|[FF D8 FF E0 00 1...|
|s3://p8-data-sora...|2024-09-11 21:10:50|  7349|[FF D8 FF E0 00 1...|
|s3://p8-data-sora...|2024-09-11 21:10:50|  7348|[FF D8 FF E0 00 1...|
|s3://p8-data-sora...|2024-09-11 21:10:51|  7328|[FF D8 FF E0 00 1...|
+--------------------+-------------------+------+--------------------+
only showing top 5 rows

<u>Je ne conserve que le **path** de l'image et j'ajoute <br />
    une colonne contenant les **labels** de chaque image</u> :

In [7]:
images = images.withColumn('label', element_at(split(images['path'], '/'),-2))
print(images.printSchema())
print(images.select('path','label').show(5,False))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- path: string (nullable = true)
 |-- modificationTime: timestamp (nullable = true)
 |-- length: long (nullable = true)
 |-- content: binary (nullable = true)
 |-- label: string (nullable = true)

None
+---------------------------------------------------------------------+----------+
|path                                                                 |label     |
+---------------------------------------------------------------------+----------+
|s3://p8-data-soraya/jupyter/jovyan/data/Test/Watermelon/r_106_100.jpg|Watermelon|
|s3://p8-data-soraya/jupyter/jovyan/data/Test/Watermelon/r_109_100.jpg|Watermelon|
|s3://p8-data-soraya/jupyter/jovyan/data/Test/Watermelon/r_108_100.jpg|Watermelon|
|s3://p8-data-soraya/jupyter/jovyan/data/Test/Watermelon/r_107_100.jpg|Watermelon|
|s3://p8-data-soraya/jupyter/jovyan/data/Test/Watermelon/r_95_100.jpg |Watermelon|
+---------------------------------------------------------------------+----------+
only showing top 5 rows

None

### 3.7.2 Préparation du modèle

Je vais utiliser la technique du **transfert learning** pour extraire les features des images.<br />
J'ai choisi d'utiliser le modèle **MobileNetV2** pour sa rapidité d'exécution comparée <br />
à d'autres modèles comme *VGG16* par exemple.

Pour en savoir plus sur la conception et le fonctionnement de MobileNetV2, <br />
je vous invite à lire [cet article](https://towardsdatascience.com/review-mobilenetv2-light-weight-model-image-classification-8febb490e61c).

Il existe une dernière couche qui sert à classer les images <br />
selon 1000 catégories que nous ne voulons pas utiliser.<br />
L'idée dans ce projet est de récupérer le **vecteur de caractéristiques <br />
de dimensions (1,1,1280)** qui servira, plus tard, au travers d'un moteur <br />
de classification à reconnaitre les différents fruits du jeu de données.

Comme d'autres modèles similaires, **MobileNetV2**, lorsqu'on l'utilise <br />
en incluant toutes ses couches, attend obligatoirement des images <br />
de dimension (224,224,3). Nos images étant toutes de dimension (100,100,3), <br />
nous devrons simplement les **redimensionner** avant de les confier au modèle.

<u>Dans l'odre</u> :
 1. Nous chargeons le modèle **MobileNetV2** avec les poids **précalculés** <br />
    issus d'**imagenet** et en spécifiant le format de nos images en entrée
 2. Nous créons un nouveau modèle avec:
  - <u>en entrée</u> : l'entrée du modèle MobileNetV2
  - <u>en sortie</u> : l'avant dernière couche du modèle MobileNetV2

In [8]:
model = MobileNetV2(weights='imagenet',
                    include_top=True,
                    input_shape=(224, 224, 3))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

14536120/14536120 [==============================] - 1s 0us/step

In [9]:
new_model = Model(inputs=model.input,
                  outputs=model.layers[-2].output)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Affichage du résumé de notre nouveau modèle où nous constatons <br />
que <u>nous récupérons bien en sortie un vecteur de dimension (1, 1, 1280)</u> :

In [10]:
brodcast_weights = sc.broadcast(new_model.get_weights())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
new_model.summary()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                                             

Tous les workeurs doivent pouvoir accéder au modèle ainsi qu'à ses poids. <br />
Une bonne pratique consiste à charger le modèle sur le driver puis à diffuser <br />
ensuite les poids aux différents workeurs.

<u>Mettons cela sous forme de fonction</u> :

In [12]:
def model_fn():
    """
    Returns a MobileNetV2 model with top layer removed 
    and broadcasted pretrained weights.
    """
    model = MobileNetV2(weights='imagenet',
                        include_top=True,
                        input_shape=(224, 224, 3))
    for layer in model.layers:
        layer.trainable = False
    new_model = Model(inputs=model.input,
                  outputs=model.layers[-2].output)
    new_model.set_weights(brodcast_weights.value)
    return new_model

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### 3.7.3 Définition du processus de chargement des images et application <br/>de leur featurisation à travers l'utilisation de pandas UDF

Ce notebook définit la logique par étapes, jusqu'à Pandas UDF.

<u>L'empilement des appels est la suivante</u> :

- Pandas UDF
  - featuriser une série d'images pd.Series
   - prétraiter une image

In [13]:
def preprocess(content):
    """
    Preprocesses raw image bytes for prediction.
    """
    img = Image.open(io.BytesIO(content)).resize([224, 224])
    arr = img_to_array(img)
    return preprocess_input(arr)

def featurize_series(model, content_series):
    """
    Featurize a pd.Series of raw images using the input model.
    :return: a pd.Series of image features
    """
    input = np.stack(content_series.map(preprocess))
    preds = model.predict(input)
    # For some layers, output features will be multi-dimensional tensors.
    # We flatten the feature tensors to vectors for easier storage in Spark DataFrames.
    output = [p.flatten() for p in preds]
    return pd.Series(output)

@pandas_udf('array<float>', PandasUDFType.SCALAR_ITER)
def featurize_udf(content_series_iter):
    '''
    This method is a Scalar Iterator pandas UDF wrapping our featurization function.
    The decorator specifies that this returns a Spark DataFrame column of type ArrayType(FloatType).

    :param content_series_iter: This argument is an iterator over batches of data, where each batch
                              is a pandas Series of image data.
    '''
    # With Scalar Iterator pandas UDFs, we can load the model once and then re-use it
    # for multiple data batches.  This amortizes the overhead of loading big models.
    model = model_fn()
    for content_series in content_series_iter:
        yield featurize_series(model, content_series)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

/mnt/yarn/usercache/livy/appcache/application_1726592999992_0002/container_1726592999992_0002_01_000001/pyspark.zip/pyspark/sql/pandas/functions.py:403: UserWarning: In Python 3.6+ and Spark 3.0+, it is preferred to specify type hints for pandas UDF instead of specifying pandas UDF type which will be deprecated in the future releases. See SPARK-28264 for more details.

### 3.7.4 Exécution des actions d'extraction de features

Nous pouvons maintenant exécuter la featurisation sur l'ensemble de notre DataFrame Spark.<br />
<u>REMARQUE</u> : Cela peut prendre beaucoup de temps, tout dépend du volume de données à traiter. <br />

Notre jeu de données de **Test** contient **22688 images**. <br />

In [15]:
features_df = images.repartition(20).select(col("path"),
                                            col("label"),
                                            featurize_udf("content").alias("features")
                                           )

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
# Vérification du nombre d'images (1051 attendues) : 
print(f"Nombre d'images : {features_df.count()}")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Nombre d'images : 22688

In [17]:
# Visualisation des 5 premières lignes du DataFrame obtenu : 
features_df.show(5, truncate=True)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------+--------------------+
|                path|         label|            features|
+--------------------+--------------+--------------------+
|s3://p8-data-sora...|    Watermelon|[1.1428745, 0.149...|
|s3://p8-data-sora...|    Watermelon|[0.042799383, 0.0...|
|s3://p8-data-sora...|    Watermelon|[0.0, 0.6704617, ...|
|s3://p8-data-sora...|Pineapple Mini|[0.0, 4.2574773, ...|
|s3://p8-data-sora...|Pineapple Mini|[0.002079701, 4.6...|
+--------------------+--------------+--------------------+
only showing top 5 rows

## 3.8 Réduction dimensionnelle

### Recherche meilleur nombre de composante

In [18]:
def preprocess_pca(dataframe):
  '''
     Préparation des données :
     - transformation en vecteur dense
     - standardisation
     param : dataframe : dataframe d'images
     return : dataframe avec features vecteur dense standardisé
  '''
  
  # Préparation des données - conversion des données images en vecteur dense
  transform_vecteur_dense = udf(lambda r: Vectors.dense(r), VectorUDT())
  dataframe = dataframe.withColumn('features_vectors', transform_vecteur_dense('features'))
  
  # Standardisation obligatoire pour PCA
  scaler_std = StandardScaler(inputCol="features_vectors", outputCol="features_scaled", withStd=True, withMean=True)
  model_std = scaler_std.fit(dataframe)
  # Mise à l'échelle
  dataframe = model_std.transform(dataframe)
  
  return dataframe

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
from pyspark.ml.feature import PCA
import numpy as np
import matplotlib.pyplot as plt

# Recherche du nombre de composantes expliquant 95% de la variance
def recherche_nb_composante(dataframe, nb_comp=None):
    '''
       Recherche du nombre de composantes expliquant 95% de la variance
       param : dataframe : dataframe d'images
       param : nb_comp : nombre de composantes à tester (None pour calculer automatiquement)
       return : k nombre de composantes expliquant 95% de la variance totale
    '''
    
    # Si nb_comp n'est pas spécifié, on récupère le nombre de caractéristiques de 'features_scaled'
    if nb_comp is None:
        first_row = dataframe.select("features_scaled").first()
        nb_comp = len(first_row[0])  # Le nombre de dimensions/features dans 'features_scaled'
    
    # Vérifier que nb_comp est supérieur à 0
    if nb_comp == 0:
        raise ValueError("La colonne 'features_scaled' est vide ou mal formatée.")
    
    # Initialisation du PCA avec nb_comp
    pca = PCA(k=nb_comp, inputCol="features_scaled", outputCol="features_pca")
    
    # Entraînement du modèle PCA
    model_pca = pca.fit(dataframe)
    variance = model_pca.explainedVariance
    
    # Affichage du graphe
    plt.plot(np.arange(len(variance)) + 1, variance.cumsum(), c="red", marker='o')
    plt.xlabel("Nb composantes")
    plt.ylabel("% variance")
    plt.title("Variance cumulée expliquée par les composantes principales")
    plt.show(block=False)
    
    # Fonction pour déterminer le nombre de composantes expliquant au moins 95% de la variance
    def nb_comp_95():
        for i in range(len(variance)):
            if variance.cumsum()[i] >= 0.95:
                print("{} composantes principales expliquent au moins 95% de la variance totale".format(i + 1))
                return i + 1
        return len(variance)  # Si aucune ne dépasse 95%, on retourne le nombre total de composantes
    
    # Appel de la fonction nb_comp pour obtenir le nombre de composantes
    k = nb_comp_95()
    
    return k


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
from pyspark.ml.feature import StandardScaler


# Pré-processing (vecteur dense, standardisation)
df_pca = preprocess_pca(features_df)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
import matplotlib.pyplot as plt
# Nombre de composante expliquant 95% de la variance
n_components = recherche_nb_composante(df_pca)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

557 composantes principales expliquent au moins 95% de la variance totale

In [22]:
# Réduction de dimension PCA
# Entrainement de l'algorithme
pca = PCA(k=n_components, inputCol='features_scaled', outputCol='vectors_pca')
model_pca = pca.fit(df_pca)

# Transformation des images sur les k premières composantes
df_reduit = model_pca.transform(df_pca)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
df_reduit.show(6)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------+--------------------+--------------------+--------------------+--------------------+
|                path|         label|            features|    features_vectors|     features_scaled|         vectors_pca|
+--------------------+--------------+--------------------+--------------------+--------------------+--------------------+
|s3://p8-data-sora...|    Watermelon|[1.1428745, 0.149...|[1.14287447929382...|[1.36039756391116...|[-14.348626953426...|
|s3://p8-data-sora...|    Watermelon|[0.042799383, 0.0...|[0.04279938340187...|[-0.6780919065655...|[-12.121304293141...|
|s3://p8-data-sora...|    Watermelon|[0.0, 0.6704617, ...|[0.0,0.6704617142...|[-0.7574011216668...|[-8.5585725681675...|
|s3://p8-data-sora...|Pineapple Mini|[0.0, 4.2574773, ...|[0.0,4.2574772834...|[-0.7574011216668...|[-9.7179667579228...|
|s3://p8-data-sora...|Pineapple Mini|[0.002079701, 4.6...|[0.00207970105111...|[-0.7535473404929...|[-13.455862354246...|
|s3://p8-data-sora...|Pi

### <u>Enregistrement des données traitées au format "**parquet**"</u> :

In [37]:
# Enregistrement des données : 
df_reduit.write.mode("overwrite").parquet(PATH_Result)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## 3.9 Chargement des données enregistrées et validation du résultat

 #### Chargement des données

In [38]:
# Chargement des données depuis path_result : 
import pandas as pd


df = pd.read_parquet(PATH_Result, engine='pyarrow')
#extraire les valeurs des colonne sous format dictionnaire
df['features_vectors'] = df['features_vectors'].apply(lambda x: x.get('values', []) if isinstance(x, dict) else x)
df['features_scaled'] = df['features_scaled'].apply(lambda x: x.get('values', []) if isinstance(x, dict) else x)
df['vectors_pca'] = df['vectors_pca'].apply(lambda x: x.get('values', []) if isinstance(x, dict) else x)

print(f'Dimension de df : {df.shape}')
df.head()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Dimension de df : (22688, 6)
                                                path  ...                                        vectors_pca
0  s3://p8-data-soraya/jupyter/jovyan/data/Test/W...  ...  [-14.348626953426535, 4.166594250388851, -6.03...
1  s3://p8-data-soraya/jupyter/jovyan/data/Test/W...  ...  [-12.121304293141602, 4.0642515814322335, -7.6...
2  s3://p8-data-soraya/jupyter/jovyan/data/Test/W...  ...  [-8.558572568167506, 5.6371767311630085, -3.98...
3  s3://p8-data-soraya/jupyter/jovyan/data/Test/P...  ...  [-9.717966757922856, 4.747763693215989, 3.8473...
4  s3://p8-data-soraya/jupyter/jovyan/data/Test/P...  ...  [-13.455862354246888, 5.939733797013632, 1.248...

[5 rows x 6 columns]

In [40]:
# Validation de la dimension des pca_features : 
df.loc[0,'features'].shape

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(1280,)

In [39]:
# Validation de la dimension des pca_features : 
df.loc[0,'vectors_pca'].shape

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(557,)

<u>On valide que la dimension du vecteur de caractéristiques des images est bien de dimension 1280</u> :

### Création d'une colonne par composante

In [41]:
columns = []
nbr_composantes = 557

# Récupération des series de composates : 
for i in range(nbr_composantes): 
    columns.append(pd.Series(df['vectors_pca'].apply(lambda x: x[i]), name=f'vectors_pca_{i+1}'))

# Concaténation des colonnes au DataFrame df : 
df = pd.concat([df] + columns, axis=1)

# Suppression de pca_features : 
df = df.drop('vectors_pca', axis=1)

# Affichage : 
print(f"Dimension de df : {df.shape}")
df.head()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Dimension de df : (22688, 562)
                                                path  ... vectors_pca_557
0  s3://p8-data-soraya/jupyter/jovyan/data/Test/W...  ...       -0.132508
1  s3://p8-data-soraya/jupyter/jovyan/data/Test/W...  ...       -0.141117
2  s3://p8-data-soraya/jupyter/jovyan/data/Test/W...  ...        0.471220
3  s3://p8-data-soraya/jupyter/jovyan/data/Test/P...  ...        0.644083
4  s3://p8-data-soraya/jupyter/jovyan/data/Test/P...  ...        0.541167

[5 rows x 562 columns]

In [42]:
# Répartition par labels : 
df['label'].value_counts()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Grape Blue       328
Plum 3           304
Walnut           249
Tomato 1         246
Peach 2          246
                ... 
Cucumber Ripe    130
Tomato Maroon    127
Mangostan        102
Pear Kaiser      102
Ginger Root       99
Name: label, Length: 131, dtype: int64

### Sauvegarde des résultats

Sauvegarde du DataFrame au format CSV dans le bucket s3

In [43]:
# Enregistrement du DataFrame en tant que fichier CSV sur S3
df.to_csv(PATH_Result + '/df_results_aws_emr.csv', index=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [45]:
# Vérification de l'enregistrement : 
df = pd.read_csv(PATH_Result + '/df_results_aws_emr.csv')

# Affichage des 5 premières lignes : 
df.head()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

                                                path  ... vectors_pca_557
0  s3://p8-data-soraya/jupyter/jovyan/data/Test/W...  ...       -0.132508
1  s3://p8-data-soraya/jupyter/jovyan/data/Test/W...  ...       -0.141117
2  s3://p8-data-soraya/jupyter/jovyan/data/Test/W...  ...        0.471220
3  s3://p8-data-soraya/jupyter/jovyan/data/Test/P...  ...        0.644083
4  s3://p8-data-soraya/jupyter/jovyan/data/Test/P...  ...        0.541167

[5 rows x 562 columns]